In [1]:
import pyautogui as at
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import pandas as pd
import requests
import bs4

In [2]:
class webdriverGet():
    extension_path1 = './LogPrograme/3.0.8_0.crx'
    extension_path2 = './LogPrograme/13.0.10_0.crx'
    download_path = "D:\VScode\github\peenutty-1688\filses_download"
    def __init__(self):
        try:
            self.web_options = webdriver.ChromeOptions()
            self.web_options.add_extension(self.extension_path1)
            self.web_options.add_extension(self.extension_path2)
            self.web_options.add_experimental_option("prefs", {
                "download.default_directory": self.download_path,
                "download.prompt_for_download": False,
                "download.directory_upgrade": True,
                "safebrowsing.enabled": True
            })
        except:
            return {"message":"error set Webdriver","status":404}
    
    def login_1688(self):
        def click_google():
            while(True):
                try:
                    self.driver.switch_to.window(self.driver.window_handles[0])
                    self.driver.find_element(By.XPATH,f"/html/body/div[1]/div/div/div[2]/section/div[2]/a[3]/figure").click()
                    return 200
                except: pass
        def write_email():
            while(True):
                try:
                    self.driver.find_element(By.XPATH,f'//*[@id="identifierId"]').send_keys(self.my_email+Keys.ENTER)
                    return 200
                except: pass
        def write_password():
            while(True):
                try:
                    self.driver.find_element(By.XPATH,f'//*[@id="password"]/div[1]/div/div[1]/input').send_keys(self.my_password+Keys.ENTER)
                    return 200
                except:pass
        click_google()
        time.sleep(3)
        write_email()
        time.sleep(3)
        write_password()
        return {"status": 200}

    def open(self):
        try:
            self.driver = webdriver.Chrome(options=self.web_options)
        except:
            return {"message":"error open webdriver","status":404}
    
    def closeTab(self):
        tab = self.driver.window_handles
        count_tab = len(tab)
        if(count_tab>1):
            for i in range(count_tab-1,0,-1):
                self.driver.switch_to.window(self.driver.window_handles[i])
                self.driver.execute_script("window.close();")

    def ToLink(self,url):
        try:
            self.driver.switch_to.window(self.driver.window_handles[0])
            self.driver.get(url=url)
        except:
            return {"message":"error get Url","status":404}
        
    def close(self):
        try:
            self.driver.quit()
        except:
            return {"message":"error close webdriver","status":404}

    def checkZero(self,_time):
        try:
            # select 0 
            self.driver.switch_to.window(self.driver.window_handles[-1])
            self.data = self.driver.page_source
            self.soup = bs4.BeautifulSoup(self.data)
            for i in range(20,0,-1):
                time.sleep(_time)
                self.get_number = self.soup.select_one(f'body > div > div > div > div.ap-page__bd > div > div.ap-compare-products__bd > div > table > tbody:nth-child(3) > tr:nth-child(3) > td:nth-child({i+1}) > div')
                try:
                    if int(self.get_number.text) == 0:
                        self.driver.find_element(By.XPATH,f"/html/body/div/div/div/div[2]/div/div[2]/div/table/thead/tr/th[{i+1}]/div[3]").click()
                except:
                    return {"message":"ไม่พบสินค้า...","status":200}
            return {"message":"เช็ค 0 สินค้าสำเร็จ","status":200}
        except:
            return {"message":"ไม่พบหน้าที่ต้องการให้เช็ค 0","status":404}
        
    def select_item20(self,_time):
        try:
            # set web index 0
            self.driver.switch_to.window(self.driver.window_handles[0])
            # click 20 item
            for index in range(20):
                time.sleep(_time)
                self.driver.find_element(By.XPATH,f'//*[@id="ap-sbi-alibabaCN-result"]/div/div[2]/div/div[1]/div[2]/div/div/div/div/div[{index+1}]/div/div[10]').click()
                self.element = self.driver.find_element(By.XPATH, f'//*[@id="ap-sbi-alibabaCN-result"]/div/div[2]/div/div[1]/div[2]/div/div/div/div/div[{index+2}]')
                self.driver.execute_script("arguments[0].scrollIntoView();", self.element)
            time.sleep(_time)
        except:
            return {"message":"error select item","status":404}

    def DownloadExcel(self):
        try:
            # 
            # download excel
            self.driver.find_element(By.XPATH,'/html/body/div/div/div/div[2]/div/div[1]/div/div').click()
        except:
            return {"message":"error Download file","status":404}
    
    def saveUrl(self,name):
        try:
            self.driver.switch_to.window(self.driver.window_handles[-1])
            open(f'{name}.txt','w',encoding='utf-8').write(self.driver.current_url)
        except:pass

<>:4: SyntaxWarning: invalid escape sequence '\V'
<>:4: SyntaxWarning: invalid escape sequence '\V'
C:\Users\Passa\AppData\Local\Temp\ipykernel_14640\337097815.py:4: SyntaxWarning: invalid escape sequence '\V'
  download_path = "D:\VScode\github\peenutty-1688\filses_download"


หลังจากที่เปิดเว็บให้ทำการล็อคอิน 1688 ก่อนถึงจะสามารถใช้งานต่อได้

In [11]:
extension_path1 = './LogPrograme/3.0.8_0.crx'
extension_path2 = './LogPrograme/13.0.10_0.crx'
download_path = "D:\VScode\github\peenutty-1688\filses_download"
web_options = webdriver.ChromeOptions()
web_options.add_extension(extension_path1)
web_options.add_extension(extension_path2)
web_options.add_experimental_option("prefs", {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})
driver = webdriver.Chrome(options=web_options)
driver.get('https://www.aliprice.com/Member/login.html?ext_id=10100&platform=1688&version=3.0.8&browser=chrome&channel=chrome&mv=3&redirect=https%3A%2F%2Fwww.1688.com')

<>:3: SyntaxWarning: invalid escape sequence '\V'
<>:3: SyntaxWarning: invalid escape sequence '\V'
C:\Users\Passa\AppData\Local\Temp\ipykernel_27200\3666168345.py:3: SyntaxWarning: invalid escape sequence '\V'
  download_path = "D:\VScode\github\peenutty-1688\filses_download"


In [54]:
# Login 1688
# email : tmhumr1frz2vjv7nf1i62lc@sharklasers.com
# password : passapol47

In [12]:
# driver.window_handles
driver.switch_to.window(driver.window_handles[-1])
driver.execute_script("window.close();")
time.sleep(1)
driver.switch_to.window(driver.window_handles[-1])
driver.execute_script("window.close();")
driver.switch_to.window(driver.window_handles[0])

In [ ]:
driver.get('chrome-extension://lenipkahddombkldhfmcnnjakmcepdlk/options.html')

In [13]:
url = 'http://localhost:3030/'
img = 'https://down-th.img.susercontent.com/file/th-11134207-7r98z-lo1hlrzwom7b75'
driver.get(f'{url}?url={img}')

In [14]:
position_img = (450,494)
downcount = 8
at.click(position_img)
at.rightClick()
time.sleep(1)
for i in range(downcount):
    at.press('down')
at.press('enter')
at.press('enter')

In [15]:
driver.switch_to.window(driver.window_handles[0])
for index in range(20):
    driver.find_element(By.XPATH,f'//*[@id="ap-sbi-alibabaCN-result"]/div/div[2]/div/div[1]/div[2]/div/div/div/div/div[{index+1}]/div/div[10]').click()
    element = driver.find_element(By.XPATH, f'//*[@id="ap-sbi-alibabaCN-result"]/div/div[2]/div/div[1]/div[2]/div/div/div/div/div[{index+2}]')
    driver.execute_script("arguments[0].scrollIntoView();", element)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ap-sbi-alibabaCN-result"]/div/div[2]/div/div[1]/div[2]/div/div/div/div/div[1]/div/div[10]"}
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6FF417012+3522402]
	(No symbol) [0x00007FF6FF038352]
	(No symbol) [0x00007FF6FEEE5ABB]
	(No symbol) [0x00007FF6FEF2BF0E]
	(No symbol) [0x00007FF6FEF2C08C]
	(No symbol) [0x00007FF6FEF6E437]
	(No symbol) [0x00007FF6FEF4F09F]
	(No symbol) [0x00007FF6FEF6BDA3]
	(No symbol) [0x00007FF6FEF4EE03]
	(No symbol) [0x00007FF6FEF1F4D4]
	(No symbol) [0x00007FF6FEF205F1]
	GetHandleVerifier [0x00007FF6FF449B9D+3730157]
	GetHandleVerifier [0x00007FF6FF49F02D+4079485]
	GetHandleVerifier [0x00007FF6FF4975D3+4048163]
	GetHandleVerifier [0x00007FF6FF16A649+718233]
	(No symbol) [0x

In [16]:
position_clicknext = (975,1002)
at.click(position_clicknext)
time.sleep(30);

In [62]:
driver.find_element(By.XPATH,'/html/body/div/div/div/div[2]/div/div[1]/div/div').click()